In [25]:
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf
from technical_indicator import RSI_calculator, MACD_calculator, momentum_calculator

# === CONFIG ===
STOCK = "CRCL"
INTERVAL = "1d"
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=60)

INITIAL_CASH   = 100_000.0
INITIAL_SHARES = 0
BLOCK_SIZE     = 100            # shares per trade
RSI_BUY_LVL    = 40
RSI_SELL_LVL   = 60
STOP_PCT       = 0.2          # 5% stop-loss from lot's entry price
FEE_PER_TRADE  = 0.00
SLIPPAGE_BPS   = 0.0            # e.g., 10 = 0.10%

# === DATA ===
df = yf.download(
    STOCK, start=START_DATE, end=END_DATE,
    interval=INTERVAL, auto_adjust=True, prepost=False, progress=False
)
df.index = pd.to_datetime(df.index)

# === INDICATORS ===
rsi_series = RSI_calculator(df)
macd_line, macd_signal = MACD_calculator(df)   # not used for signals but available
momentum_series = momentum_calculator(df)

ind = pd.concat(
    [df["Open"], df["Close"], rsi_series, macd_line, macd_signal, momentum_series],
    axis=1
).dropna()
ind.columns = ["Open", "Close", "RSI", "MACD", "MACD_Signal", "Momentum"]

# --- Build yesterday's signals; execute at today's OPEN (next-bar execution)
mom_up   = (ind["Momentum"] > 0) | (ind["Momentum"].shift(1) <= 0)
mom_down = (ind["Momentum"] < 0)

buy_signal_prev  = (ind["RSI"] < RSI_BUY_LVL) & mom_up
sell_signal_prev = (ind["RSI"] > RSI_SELL_LVL) | mom_down

buy_exec  = buy_signal_prev.shift(1).fillna(False)
sell_exec = sell_signal_prev.shift(1).fillna(False)

# === BACKTEST with per-lot 5% stop-loss ===
cash   = float(INITIAL_CASH)
shares = int(INITIAL_SHARES)
log = []

# Track lots: list of dicts with {"qty": int, "entry": float}
lots = []
slip = SLIPPAGE_BPS / 10_000.0

opens  = ind["Open"].astype(float)
closes = ind["Close"].astype(float)

for i in range(1, len(ind)):
    ts = ind.index[i]
    px_open = opens.iat[i]

    # 1) STOP-LOSS check first (sell ONE breached lot if any)
    #    If you prefer to sell ALL breached lots, loop until no breaches remain.
    stop_sold = False
    for li, lot in enumerate(lots):
        trigger_price = lot["entry"] * (1.0 - STOP_PCT)
        if px_open <= trigger_price and lot["qty"] > 0:
            qty = min(BLOCK_SIZE, lot["qty"])
            proceeds = qty * px_open * (1 - slip) - FEE_PER_TRADE
            cash  += proceeds
            shares -= qty
            lot["qty"] -= qty
            log.append((ts, "STOP_SELL", qty, px_open, shares, cash, lot["entry"], trigger_price))
            stop_sold = True
            break  # sell only one lot per bar on stop; remove this break to sell all
    if stop_sold:
        continue  # skip discretionary buys/sells this bar after a stop triggers

    # 2) Discretionary BUY
    if buy_exec.iat[i]:
        cost = BLOCK_SIZE * px_open * (1 + slip) + FEE_PER_TRADE
        if cash >= cost:
            cash  -= cost
            shares += BLOCK_SIZE
            lots.append({"qty": BLOCK_SIZE, "entry": px_open})
            log.append((ts, "BUY", BLOCK_SIZE, px_open, shares, cash, px_open, None))
        # else: not enough cash -> skip

    # 3) Discretionary SELL
    elif sell_exec.iat[i] and shares >= BLOCK_SIZE:
        # Sell from oldest lot (FIFO)
        sell_qty = BLOCK_SIZE
        px = px_open * (1 - slip)
        cash += sell_qty * px - FEE_PER_TRADE
        shares -= sell_qty

        # decrement from lots FIFO
        remaining = sell_qty
        for lot in lots:
            if lot["qty"] == 0:
                continue
            used = min(lot["qty"], remaining)
            lot["qty"] -= used
            remaining -= used
            if remaining == 0:
                break

        log.append((ts, "SELL", sell_qty, px_open, shares, cash, None, None))

# Final portfolio stats
last_price = closes.iloc[-1]
final_value = cash + shares * last_price
start_value = INITIAL_CASH + INITIAL_SHARES * closes.iloc[0]
pnl = final_value - start_value
roi = (pnl / start_value) * 100 if start_value else 0.0

# === RESULTS ===
cols = ["Time", "Action", "Qty", "ExecPrice", "Shares_After", "Cash_After", "LotEntry", "StopTrigger"]
trades = pd.DataFrame(log, columns=cols)

print("\n=== SUMMARY ===")
print(f"Start Value:        ${start_value:,.2f}")
print(f"Final Cash:         ${cash:,.2f}")
print(f"Final Shares:       {shares}")
print(f"Last Price:         ${last_price:,.2f}")
print(f"Final Portfolio:    ${final_value:,.2f}")
print(f"P&L:                ${pnl:,.2f}  ({roi:.2f}%)")
print(f"Total Trades:       {len(trades)}")

print("\n=== TRADE LOG (last 10) ===")
print(trades.tail(10).to_string(index=False))


=== SUMMARY ===
Start Value:        $100,000.00
Final Cash:         $57,800.50
Final Shares:       300
Last Price:         $147.23
Final Portfolio:    $101,969.98
P&L:                $1,969.98  (1.97%)
Total Trades:       9

=== TRADE LOG (last 10) ===
      Time Action  Qty  ExecPrice  Shares_After   Cash_After   LotEntry StopTrigger
2025-08-05    BUY  100 160.000000           100 84000.000000 160.000000        None
2025-08-06    BUY  100 150.089996           200 68991.000366 150.089996        None
2025-08-07    BUY  100 167.679993           300 52223.001099 167.679993        None
2025-08-08    BUY  100 155.014999           400 36721.501160 155.014999        None
2025-08-11   SELL  100 161.005005           300 52822.001648        NaN        None
2025-08-12   SELL  100 186.300003           200 71452.001953        NaN        None
2025-08-13   SELL  100 156.639999           100 87116.001892        NaN        None
2025-08-14    BUY  100 152.000000           200 71916.001892 152.000000   

/var/folders/sr/t9b4x2mn7s35cskh4j9lc0qh0000gn/T/ipykernel_73425/995673229.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  buy_exec  = buy_signal_prev.shift(1).fillna(False)
/var/folders/sr/t9b4x2mn7s35cskh4j9lc0qh0000gn/T/ipykernel_73425/995673229.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sell_exec = sell_signal_prev.shift(1).fillna(False)


In [23]:
ind

,Open,Close,RSI,MACD,MACD_Signal,Momentum
Date,,,,,,
2025-07-08,297.000000,297.809998,20.604179,-6.624128,-3.304642,-31.320007
2025-07-09,297.549988,295.880005,20.322186,-7.565148,-4.156743,-20.470001
2025-07-10,300.049988,309.869995,28.013179,-7.100191,-4.745433,-12.179993
2025-07-11,307.890015,313.510010,29.908916,-6.364624,-5.069271,-8.649994
2025-07-14,317.730011,316.899994,31.712538,-5.445367,-5.144490,-31.779999
2025-07-15,319.679993,310.779999,30.201586,-5.151300,-5.145852,-29.690002
2025-07-16,312.799988,321.670013,36.041172,-3.993483,-4.915378,-5.879974
2025-07-17,323.149994,319.410004,35.379631,-3.221137,-4.576530,-6.369995
2025-07-18,321.660004,329.649994,40.691510,-1.762450,-4.013714,6.019989
